In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
import pprint
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from collections import OrderedDict

from dataset import *
from runner import ExptRunner
from networks.imageencoder import *
from networks.imagedecoder import *
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.dense import *
from networks.lossfunctions import *
from networks.multinet import *
from networks.special import *
from helpers import *
import utils
import simplereacherdimensions
import simplereacheradapters
from env.simulator import simulator, evaluator

In [3]:
import warnings
warnings.filterwarnings("ignore", category=torch.serialization.SourceChangeWarning)

In [4]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [5]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [6]:
if torch.cuda.is_available():
    print (torch.cuda.current_device())
    torch.cuda.set_device(device)
    print (torch.cuda.current_device())

In [7]:
root_folder = 'data/simple_reacher/training'
# number_of_trajectories = 10
number_of_trajectories = 5000

In [8]:
start = time.time()
train_data = SimpleReacherOnDemandDataset(root_folder, range(475), device=device)
test_data = SimpleReacherOnDemandDataset(root_folder, range(475, 500), device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 0.8644099235534668 sec


In [9]:
print (len(train_data))
print (len(test_data))

10000
459


In [10]:
def run_expts(train_epochs):
    for i in range(len(expts)):
        arg_lists = expts[i]
        cons_args = arg_lists[0]
        train_args = {}
        if (len(arg_lists)) > 1:
            train_args = arg_lists[1]
        test_args = {}
        if (len(arg_lists)) > 2:
            test_args = arg_lists[2]
        
        runner = ExptRunner(train_data=train_data, test_data=test_data, device=device, **cons_args)
        print ("Experiment logs folder: {}".format(runner.log_folder))
        with open(runner.log_folder + '/expt.txt', 'w') as f:
            f.write(pprint.pformat(arg_lists))
        runner.train(train_epochs, **train_args)
        runner.test(**test_args)

In [11]:
expts = [
    [{
        "expt_prefix":'ImageAutoEncoder-Reacher-ComposedAutoEncoder-MSE',
        "net_func": lambda: ComposedAutoEncoder(x_dim=3, z_dim=4096, img_res=256,layers_channels=[16,16,16,16], useMaxPool=True, device=device),
        "data_adapter_func": simplereacheradapters.It_scaled_adapter,
        "loss_adapter_func": mse_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": l1_loss_adapter
    }],
]

In [ ]:
train_epochs = 2
run_expts(train_epochs)

Experiment logs folder: runs/07-18-14-52-15-ImageAutoEncoder-Reacher-ComposedAutoEncoder-MSE


In [98]:
testchkpt = torch.load('runs/07-20/run2/07-20-21-21-05-Policy-Reacher-Dense-MSE/train_checkpoint.tar', map_location=device)
testNet = testchkpt['model']
testNet.eval()

Dense(
  (net): Sequential(
    (pol_fc0): Linear(in_features=6, out_features=32, bias=True)
    (pol_relu0): ReLU()
    (pol_fc1): Linear(in_features=32, out_features=32, bias=True)
    (pol_relu1): ReLU()
    (pol_fc2): Linear(in_features=32, out_features=3, bias=True)
    (pol_sig2): Sigmoid()
  )
)

In [90]:
class AdapterNet(nn.Module):
    def __init__(self, testNet, ip_adapter=None, op_adapter=None):
        super().__init__()
        self.testNet = testNet
        self.ip_adapter = ip_adapter if ip_adapter else lambda x:x
        self.op_adapter = op_adapter if op_adapter else lambda x:x
        
    def forward(self, data):
        ip = self.ip_adapter(data)
        op = self.testNet(ip)
        return self.op_adapter(op)

In [91]:
t475 = test_data.trajectory_data[test_data.trajectory_index[:, 0] == 475, :]

In [92]:
t475.shape

torch.Size([19, 9])

In [99]:
ip_adapter = lambda x:x.float()
op_adapter = lambda x:((x * (2 * simplereacherdimensions.joint_max)) - (simplereacherdimensions.joint_max))
ev = evaluator(AdapterNet(testNet, ip_adapter=ip_adapter, op_adapter=op_adapter))

In [100]:
t475_output = ev.rollout(t475)

Creating env


In [101]:
t475_output

tensor([[2.0023, 0.4158, 1.8592, 0.9967, 1.1594, 2.5409],
        [1.9043, 0.5139, 1.7611, 0.9996, 1.1593, 2.5384],
        [1.8053, 0.5956, 1.6621, 1.0007, 1.1592, 2.5375],
        [1.7063, 0.5854, 1.7592, 1.0011, 1.1592, 2.5371],
        [1.6073, 0.6034, 1.8582, 1.0012, 1.1592, 2.5370],
        [1.5083, 0.6397, 1.9572, 1.0013, 1.1592, 2.5369],
        [1.4093, 0.7053, 2.0562, 1.0013, 1.1592, 2.5369],
        [1.3102, 0.7835, 2.1552, 1.0013, 1.1592, 2.5369],
        [1.2112, 0.8817, 2.2542, 1.0013, 1.1592, 2.5369],
        [1.1122, 0.9807, 2.2044, 1.0013, 1.1592, 2.5369],
        [1.0132, 1.0797, 2.2021, 1.0013, 1.1592, 2.5369],
        [0.9142, 1.1061, 2.3001, 1.0013, 1.1592, 2.5369],
        [0.8152, 1.0081, 2.3929, 1.0013, 1.1592, 2.5369],
        [0.7162, 0.9091, 2.3035, 1.0013, 1.1592, 2.5369],
        [0.6172, 0.8319, 2.2383, 1.0013, 1.1592, 2.5369],
        [0.5182, 0.7456, 2.1949, 1.0013, 1.1592, 2.5369],
        [0.4192, 0.6712, 2.1471, 1.0013, 1.1592, 2.5369],
        [0.320

In [96]:
t475[:, :6]

tensor([[2.0023, 0.4158, 1.8592, 0.9967, 1.1594, 2.5409],
        [1.9702, 0.3573, 1.9128, 0.9967, 1.1594, 2.5409],
        [1.9658, 0.3536, 1.9160, 0.9967, 1.1594, 2.5409],
        [1.9073, 0.4041, 1.9532, 0.9967, 1.1594, 2.5409],
        [1.8453, 0.4557, 1.9929, 0.9967, 1.1594, 2.5409],
        [1.7844, 0.5060, 2.0321, 0.9967, 1.1594, 2.5409],
        [1.7239, 0.5562, 2.0713, 0.9967, 1.1594, 2.5409],
        [1.6633, 0.6065, 2.1104, 0.9967, 1.1594, 2.5409],
        [1.6027, 0.6567, 2.1495, 0.9967, 1.1594, 2.5409],
        [1.5421, 0.7070, 2.1887, 0.9967, 1.1594, 2.5409],
        [1.4815, 0.7573, 2.2278, 0.9967, 1.1594, 2.5409],
        [1.4209, 0.8075, 2.2670, 0.9967, 1.1594, 2.5409],
        [1.3603, 0.8578, 2.3061, 0.9967, 1.1594, 2.5409],
        [1.2997, 0.9080, 2.3452, 0.9967, 1.1594, 2.5409],
        [1.2391, 0.9583, 2.3844, 0.9967, 1.1594, 2.5409],
        [1.1785, 1.0086, 2.4235, 0.9967, 1.1594, 2.5409],
        [1.1179, 1.0588, 2.4627, 0.9967, 1.1594, 2.5409],
        [1.057

In [102]:
print (F.l1_loss(t475[:, :3], t475_output[:, :3]))
print (F.l1_loss(t475[-1:, :3], t475_output[-1:, :3]))

tensor(0.2655, dtype=torch.float64)
tensor(0.6274, dtype=torch.float64)
